In [ ]:
from sagemaker.chainer import Chainer
from sagemaker import get_execution_role
import sagemaker
role = get_execution_role()

bucket_name ="<bucketname>"

sagemaker_session = sagemaker.Session()

sourcecode_s3_uri = "<s3://mybucket/mycode.zip>"

This launches the job on a device with no cpu.. hence slow ..

In [ ]:
%%time
import boto3
from time import gmtime, strftime

job_name = 'Chainer-lowlevel-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("Training job", job_name)

images = {  'us-east-2': '520713654638.dkr.ecr.us-east-2.amazonaws.com/sagemaker-chainer:4.0.0-gpu-py3',
           }
image = images[boto3.Session().region_name]

output_location = 's3://{}/sentimentanalysis/output'.format(bucket_name)
print('training artifacts will be uploaded to: {}'.format(output_location))

create_training_params = \
{
    "AlgorithmSpecification": {
        "TrainingImage": image,
        "TrainingInputMode": "File"
    },
    "RoleArn": role,
    "OutputDataConfig": {
        "S3OutputPath": output_location
    },
    "ResourceConfig": {
        "InstanceCount": 1,
        "InstanceType": "ml.p3.16xlarge",
        "VolumeSizeInGB": 70
    },
    "TrainingJobName": job_name,
    "HyperParameters": {'epoch': '10', 
                        'batchsize': '50', 
                        'traindata':'yelp_reviews.train.shuffled.csv', 
                        'testdata':'yelp_reviews.test.shuffled.csv',
                        'sagemaker_program': "main_full_cycle.py",
                        'sagemaker_region':'us-east-2',
                        'gpu': '0',
                        'sagemaker_submit_directory': sourcecode_s3_uri
                       

               ,"sagemaker_job_name"         :job_name 
                       ,'pretrained-embed':'glove.6B.300d.txt'
                       },
    "StoppingCondition": {
        "MaxRuntimeInSeconds": 60 * 60 * 24 * 5
    },
    "InputDataConfig": [
        {
            "ChannelName": "train",
            
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "S3Prefix",
                    "S3Uri": 's3://{}/yelp_reviews.train.shuffled.csv'.format(bucket_name),
                    "S3DataDistributionType": "FullyReplicated"
                }
            },
            "CompressionType": "None",
            "RecordWrapperType": "None"
        }
        ,{
            "ChannelName": "test",
            
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "S3Prefix",
                    "S3Uri": 's3://{}/yelp_reviews.test.shuffled.csv'.format(bucket_name),
                    "S3DataDistributionType": "FullyReplicated"
                }
            },
            "CompressionType": "None",
            "RecordWrapperType": "None"
        },
        {
           "ChannelName" :'PRETRAINEDEMBED'
            ,"DataSource": {
                "S3DataSource": {
                    "S3DataType": "S3Prefix",
                    "S3Uri": 's3://{}/glove.6B.300d.txt'.format(bucket_name),
                    "S3DataDistributionType": "FullyReplicated"
                }
            },
            "CompressionType": "None",
            "RecordWrapperType": "None"
            
        }
    ]
}



sagemaker = boto3.client('sagemaker')

sagemaker.create_training_job(**create_training_params)

status = sagemaker.describe_training_job(TrainingJobName=job_name)['TrainingJobStatus']
print(status)

try:
    sagemaker.get_waiter('training_job_completed_or_stopped').wait(TrainingJobName=job_name)
finally:
    status = sagemaker.describe_training_job(TrainingJobName=job_name)['TrainingJobStatus']
    print("Training job ended with status: " + status)
    if status == 'Failed':
        message = sagemaker.describe_training_job(TrainingJobName=job_name)['FailureReason']
        print('Training failed with the following error: {}'.format(message))
        raise Exception('Training job failed')

In [ ]:
import boto3
from urllib.parse import urlparse

def rename_object(s3_source_url, s3_destination_url):      
    s = urlparse(s3_source_url)
    d = urlparse(s3_destination_url)
    s3 = boto3.resource('s3')
    s3.Object(d.netloc, d.path.lstrip('/')).copy_from(CopySource=s.netloc+s.path)

In [ ]:
chainer_training_job = chainer_estimator.latest_training_job.name
desc = sagemaker_session.sagemaker_client.describe_training_job(TrainingJobName=chainer_training_job)
model_data = desc['ModelArtifacts']['S3ModelArtifacts'] #.replace('model.tar.gz', 'output.tar.gz')
output_data = model_data.replace('model.tar.gz', 'output.tar.gz')

rename_object(output_data, model_data)
#retrieve_output_from_s3(output_data, 'output/sentiment')

In [ ]:
predictor = chainer_estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')